In [1]:
# see CDAS git /Raid/src/Utils/CL/cl_msg_unknown_pack.h
def read_package(f) -> dict:

    package = {}

    # pack Header
    package['LsId'] = int.from_bytes(f.read(2), byteorder='little')
    n_packs = int.from_bytes(f.read(2), byteorder='little')
    timestamp_sec = int.from_bytes(f.read(4), byteorder='little')
    timestamp_nsec = int.from_bytes(f.read(4), byteorder='little')
    total_size = int.from_bytes(f.read(4), byteorder='little')

    for i in range(n_packs):

        # header i
        type = int.from_bytes(f.read(4), byteorder='little')
        version = int.from_bytes(f.read(4), byteorder='little')
        size = int.from_bytes(f.read(4), byteorder='little')
        data = f.read(size)

        package[str(i)] = {'type': type,
                        'version': version,
                        'size': size,
                        'data': data}

    return package

examples = [] 
with open('/home/filip/Desktop/didi-synvc/monit_2024_11_21.packs.nobackup', 'rb') as f:
    while f.read(8) == b'!monit!!':
        examples.append(read_package(f))

In [ ]:
for e in examples:
        print(e)

{'LsId': 136, '0': {'type': 3, 'version': 1, 'size': 4, 'data': b'\xce\xda\xceB'}, '1': {'type': 4, 'version': 1, 'size': 12, 'data': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}}
{'LsId': 136, '0': {'type': 3, 'version': 1, 'size': 4, 'data': b'D=\xceB'}, '1': {'type': 4, 'version': 1, 'size': 12, 'data': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}}
{'LsId': 136, '0': {'type': 3, 'version': 1, 'size': 4, 'data': b'\xce\xda\xccB'}, '1': {'type': 4, 'version': 1, 'size': 12, 'data': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}}
{'LsId': 136, '0': {'type': 3, 'version': 1, 'size': 4, 'data': b'?\xf7\xd1B'}, '1': {'type': 4, 'version': 1, 'size': 12, 'data': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}}
{'LsId': 136, '0': {'type': 3, 'version': 1, 'size': 4, 'data': b'\r\xd2\xd4B'}, '1': {'type': 4, 'version': 1, 'size': 12, 'data': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}}
{'LsId': 136, '0': {'type': 3, 'version': 1, 'size': 4, 'data': b'\xec\